# Quantum Machine Learning with PennyLane

This notebook demonstrates devqubit tracking for **QML classification experiments**.

**What you'll see:**
- Two encoding architectures: **Angle Encoding** vs **IQP Encoding**
- Training curves (loss, accuracy)
- Architecture comparison with bar chart
- Decision boundary visualization

**The problem:** Binary classification of 2D points (inside vs outside a circle).

In [ ]:
from importlib.metadata import entry_points

if not any(ep.name == "pennylane" for ep in entry_points(group="devqubit.adapters")):
    raise ImportError(
        "devqubit Pennylane adapter is not installed.\n"
        "Install with: pip install 'devqubit[pennylane]'"
    )
else:
    print("PennyLane adapter available!")

In [ ]:
from pathlib import Path
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as pnp

from devqubit import Config, set_config, track
from devqubit.compare import diff
from devqubit.runs import load_run, list_groups, list_runs_in_group

### Setup

We use 3-qubit variational classifiers with two encoding strategies:

| Architecture | Encoding | Variational Part |
|--------------|----------|------------------|
| **Angle Encoding** | RY/RZ rotations (data re-uploading) | Rot gates per layer |
| **IQP Encoding** | Hadamard + RZ + IsingZZ interactions | StronglyEntanglingLayers |

In [ ]:
# Configuration
PROJECT = "qml_classification"
N_QUBITS = 3
N_SAMPLES = 500
N_EPOCHS = 50
LR = 0.3
SEED = 42

# Workspace
WORKSPACE = Path(".devqubit_qml_demo")
if WORKSPACE.exists():
    shutil.rmtree(WORKSPACE)

set_config(Config(root_dir=WORKSPACE))

np.random.seed(SEED)
pnp.random.seed(SEED)
print(f"Workspace: {WORKSPACE.resolve()}")

### 1. Dataset: Circle Classification

Binary classification of 2D points:
- **Class 0**: outside circle (radius > 0.5)
- **Class 1**: inside circle (radius < 0.5)

This is a classic non-linear classification problem that quantum classifiers can handle well.

In [ ]:
def generate_circle_data(n_samples: int, seed: int):
    """Generate 2D binary classification data."""
    rng = np.random.default_rng(seed)
    n = n_samples // 2

    # Class 1: inside
    θ_in = rng.uniform(0, 2 * np.pi, n)
    r_in = rng.uniform(0, 0.45, n)
    X_in = np.column_stack([r_in * np.cos(θ_in), r_in * np.sin(θ_in)])

    # Class 0: outside
    θ_out = rng.uniform(0, 2 * np.pi, n)
    r_out = rng.uniform(0.6, 1.0, n)
    X_out = np.column_stack([r_out * np.cos(θ_out), r_out * np.sin(θ_out)])

    X = np.vstack([X_out, X_in])
    y = np.hstack([np.zeros(n), np.ones(n)])
    perm = rng.permutation(len(y))
    return X[perm], y[perm].astype(int)


def train_test_split(X, y, test_frac=0.2, seed=0):
    rng = np.random.default_rng(seed)
    idx = rng.permutation(len(y))
    n_test = int(len(y) * test_frac)
    return X[idx[n_test:]], X[idx[:n_test]], y[idx[n_test:]], y[idx[:n_test]]

In [ ]:
X, y = generate_circle_data(N_SAMPLES, SEED)
X_train, X_test, y_train, y_test = train_test_split(X, y, 0.2, SEED)

print(f"Training samples: {len(X_train)}")
print(f"Test samples:     {len(X_test)}")
print(f"Class balance:    {y_train.mean():.1%} positive")

In [ ]:
# Visualize dataset
fig, ax = plt.subplots(figsize=(5, 5))
ax.scatter(
    X_train[y_train == 0, 0],
    X_train[y_train == 0, 1],
    c="tab:blue",
    s=15,
    alpha=0.6,
    label="Outside",
)
ax.scatter(
    X_train[y_train == 1, 0],
    X_train[y_train == 1, 1],
    c="tab:orange",
    s=15,
    alpha=0.6,
    label="Inside",
)
ax.add_patch(plt.Circle((0, 0), 0.5, fill=False, linestyle="--", color="gray"))
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.set_aspect("equal")
ax.legend()
ax.set_title("Training Data")
plt.show()

### 2. Two Classifier Architectures

- Angle Encoding (Data Re-uploading): simple rotations encode features at each layer, good for learning radial patterns.
- IQP Encoding: uses ZZ interactions to encode feature products, can capture more complex correlations.

We measure ⟨Z⟩ and map it to class probabilities.

In [ ]:
def create_angle_classifier(n_layers, dev):
    """Data re-uploading classifier with angle encoding."""

    @qml.qnode(dev, diff_method="backprop")
    def circuit(x, params):
        x0, x1 = x[..., 0], x[..., 1]
        r2 = x0**2 + x1**2  # radial feature

        for layer in range(n_layers):
            # Re-upload features
            qml.RY(pnp.pi * x0, wires=0)
            qml.RZ(pnp.pi * x1, wires=0)
            qml.RY(pnp.pi * r2, wires=0)
            # Trainable rotation
            qml.Rot(params[layer, 0], params[layer, 1], params[layer, 2], wires=0)

        return qml.expval(qml.PauliZ(0))

    return circuit, (n_layers, 3)


def create_iqp_classifier(n_layers, dev):
    """IQP-style encoding with entangling variational layers."""

    @qml.qnode(dev, diff_method="backprop")
    def circuit(x, params):
        x0, x1 = x[..., 0], x[..., 1]
        r2 = x0**2 + x1**2

        # IQP encoding
        for w in range(3):
            qml.Hadamard(wires=w)
        qml.RZ(pnp.pi * x0, wires=0)
        qml.RZ(pnp.pi * x1, wires=1)
        qml.RZ(pnp.pi * r2, wires=2)

        # Feature interactions
        qml.IsingZZ(pnp.pi * x0 * x1, wires=[0, 1])
        qml.IsingZZ(pnp.pi * x1 * r2, wires=[1, 2])
        qml.IsingZZ(pnp.pi * x0 * r2, wires=[0, 2])

        # Variational layers
        qml.StronglyEntanglingLayers(params, wires=[0, 1, 2])

        return qml.expval(qml.PauliZ(0))

    return circuit, (n_layers, 3, 3)

### 3. Training function

We define a training function that:
- Optimizes the variational parameters using gradient descent
- Logs loss and accuracy at each epoch
- Tracks the best model by validation accuracy

This mirrors typical ML training loops but with quantum circuit evaluation.

In [ ]:
def train_classifier(circuit, param_shape, run):
    """Train and log metrics to devqubit."""
    X_tr, y_tr = pnp.asarray(X_train), pnp.asarray(y_train)
    X_te, y_te = pnp.asarray(X_test), pnp.asarray(y_test)

    params = pnp.random.uniform(-np.pi, np.pi, param_shape, requires_grad=True)
    opt = qml.AdamOptimizer(stepsize=LR)

    def predict(X, p):
        return (circuit(X, p) + 1) / 2

    def loss_fn(p):
        preds = pnp.clip(predict(X_tr, p), 1e-7, 1 - 1e-7)
        return -pnp.mean(y_tr * pnp.log(preds) + (1 - y_tr) * pnp.log(1 - preds))

    def accuracy(p, X, y):
        preds = np.asarray(predict(X, p))
        return float(np.mean((preds > 0.5) == np.asarray(y)))

    history = {"loss": [], "train_acc": [], "test_acc": []}
    best_acc, best_params = 0, params

    for epoch in range(N_EPOCHS):
        params, loss = opt.step_and_cost(loss_fn, params)
        train_acc = accuracy(params, X_tr, y_tr)
        test_acc = accuracy(params, X_te, y_te)

        history["loss"].append(float(loss))
        history["train_acc"].append(train_acc)
        history["test_acc"].append(test_acc)

        run.log_metric("loss", float(loss), step=epoch)
        run.log_metric("train_acc", train_acc, step=epoch)
        run.log_metric("test_acc", test_acc, step=epoch)

        if test_acc > best_acc:
            best_acc, best_params = test_acc, params

    return best_params, best_acc, history

### 4. Baseline Run

Train a 2-layer classifier as our baseline. We log:
- Architecture parameters
- Epoch-by-epoch metrics
- Final trained weights as an artifact

In [ ]:
with track(project=PROJECT, run_name="baseline_angle") as run:
    dev = qml.device("default.qubit", wires=N_QUBITS)
    tracked_dev = run.wrap(dev)

    circuit, param_shape = create_angle_classifier(2, tracked_dev)

    run.log_params(
        {
            "architecture": "angle_encoding",
            "n_layers": 2,
            "n_qubits": N_QUBITS,
            "lr": LR,
        }
    )
    run.set_tags({"role": "baseline"})

    print("Training Angle Encoding (2 layers)...")
    best_params, best_acc, baseline_history = train_classifier(
        circuit, param_shape, run
    )

    run.log_metrics({"best_test_acc": best_acc})
    run.log_json(
        "trained_params",
        {"values": np.asarray(best_params).tolist()},
        role="model",
    )
    baseline_id = run.run_id

print(f"Baseline accuracy: {best_acc:.1%}")

In [ ]:
# Training curves
fig, axes = plt.subplots(1, 2, figsize=(10, 4))

axes[0].plot(baseline_history["loss"], "o-", markersize=3)
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")
axes[0].set_title("Training Loss")
axes[0].grid(True, alpha=0.3)

axes[1].plot(baseline_history["train_acc"], "o-", markersize=3, label="Train")
axes[1].plot(baseline_history["test_acc"], "s-", markersize=3, label="Test")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy")
axes[1].set_title("Accuracy")
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Architecture Sweep

How many variational layers do we need? Which ansatz is better for our problem? Let's sweep over different circuits and depths to compare.

Using `group_id` keeps these runs organized together. All runs:
- Share the same group for easy querying
- Use identical random seed for fair comparison
- Vary only the number of layers

In [ ]:
architectures = [
    ("angle_encoding", create_angle_classifier, 2),
    ("angle_encoding", create_angle_classifier, 3),
    ("iqp_encoding", create_iqp_classifier, 2),
    ("iqp_encoding", create_iqp_classifier, 3),
]

sweep_results = []
sweep_params = {}  # Store for decision boundary

print("Architecture Sweep")
print("=" * 50)

for arch_name, builder, n_layers in architectures:
    with track(
        project=PROJECT, group_id="arch_sweep", group_name="Architecture Comparison"
    ) as run:
        dev = qml.device("default.qubit", wires=N_QUBITS)
        tracked_dev = run.wrap(dev)

        circuit, param_shape = builder(n_layers, tracked_dev)
        n_params = int(np.prod(param_shape))

        run.log_params(
            {
                "architecture": arch_name,
                "n_layers": n_layers,
                "n_params": n_params,
            }
        )
        run.set_tags({"role": "sweep"})

        print(f"  {arch_name} (L={n_layers}, params={n_params})...", end=" ")
        best_params, best_acc, _ = train_classifier(circuit, param_shape, run)

        run.log_metrics({"best_test_acc": best_acc})
        run.log_json(
            "trained_params",
            {"values": np.asarray(best_params).tolist()},
            role="model",
        )

        sweep_results.append(
            {
                "arch": arch_name,
                "layers": n_layers,
                "acc": best_acc,
                "run_id": run.run_id,
            }
        )
        sweep_params[(arch_name, n_layers)] = (best_params, builder)

        print(f"{best_acc:.1%}")

best = max(sweep_results, key=lambda r: r["acc"])
print(f"\n=> Best: {best['arch']} L={best['layers']} ({best['acc']:.1%})")

In [ ]:
# Bar chart comparison
labels = [
    f"{r['arch'].replace('_encoding','')}\nL={r['layers']}" for r in sweep_results
]
accs = [r["acc"] for r in sweep_results]
colors = ["tab:blue" if "angle" in r["arch"] else "tab:orange" for r in sweep_results]

fig, ax = plt.subplots(figsize=(8, 4))
bars = ax.bar(labels, accs, color=colors, edgecolor="black", alpha=0.8)
ax.set_ylabel("Test Accuracy")
ax.set_title("Architecture Comparison")
ax.set_ylim(0, 1.05)
ax.axhline(
    y=best["acc"],
    color="red",
    linestyle="--",
    alpha=0.5,
    label=f"Best: {best['acc']:.1%}",
)
ax.legend()
ax.grid(True, axis="y", alpha=0.3)

# Add value labels
for bar, acc in zip(bars, accs):
    ax.text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.02,
        f"{acc:.0%}",
        ha="center",
        va="bottom",
        fontsize=10,
    )

plt.tight_layout()
plt.show()

Let's visualize what the best model learned.

In [ ]:
# Get best model
best_params_final, best_builder = sweep_params[(best["arch"], best["layers"])]
eval_circuit, _ = best_builder(
    best["layers"], qml.device("default.qubit", wires=N_QUBITS)
)

# Create prediction grid
xx, yy = np.meshgrid(np.linspace(-1.2, 1.2, 50), np.linspace(-1.2, 1.2, 50))
grid = np.column_stack([xx.ravel(), yy.ravel()])
grid_preds = ((np.array(eval_circuit(grid, best_params_final)) + 1) / 2).reshape(
    xx.shape
)

# Plot
fig, ax = plt.subplots(figsize=(6, 5))
contour = ax.contourf(xx, yy, grid_preds, levels=20, cmap="RdYlBu", alpha=0.7)
ax.contour(xx, yy, grid_preds, levels=[0.5], colors="k", linewidths=2)
ax.scatter(
    X_test[y_test == 0, 0],
    X_test[y_test == 0, 1],
    c="tab:blue",
    s=20,
    edgecolors="k",
    linewidths=0.5,
    label="Outside",
)
ax.scatter(
    X_test[y_test == 1, 0],
    X_test[y_test == 1, 1],
    c="tab:orange",
    s=20,
    edgecolors="k",
    linewidths=0.5,
    label="Inside",
)
ax.add_patch(
    plt.Circle(
        (0, 0),
        0.5,
        fill=False,
        linestyle="--",
        color="gray",
        linewidth=1.5,
    )
)
ax.set_xlim(-1.2, 1.2)
ax.set_ylim(-1.2, 1.2)
ax.set_aspect("equal")
ax.set_title(f"Decision Boundary: {best['arch']} (L={best['layers']})")
ax.legend(loc="upper right")
plt.colorbar(contour, ax=ax, label="P(inside)")
plt.tight_layout()
plt.show()

### 6. Compare Runs

Let's compare our baseline (2 layers) with the best configuration from the sweep.

The `diff()` function shows:
- Parameter differences
- Metric differences
- Tag differences

In [ ]:
comparison = diff(baseline_id, best["run_id"])

print("Baseline vs Best Architecture")
print("=" * 45)
print(comparison)

### 8. Query Results

Browse all runs and groups to see the full experiment history.

In [ ]:
print("Experiment Summary")
print("=" * 60)

for g in list_groups():
    print(f"\n{g['group_name']}:")
    for run_info in list_runs_in_group(g["group_id"]):
        rec = load_run(run_info["run_id"])
        arch = rec.params.get("architecture", "?")
        layers = rec.params.get("n_layers", "?")
        acc = rec.metrics.get("best_test_acc", 0)
        print(f"  {arch:15s} L={layers}: {acc:.1%}")

---
## Summary

| Feature | What we tracked |
|---------|----------------|
| **Architectures** | angle_encoding vs iqp_encoding |
| **Epoch metrics** | loss, train_acc, test_acc |
| **Sweep** | 4 configurations (2 arch × 2 depths) |
| **Artifacts** | Trained parameters as JSON |
| **Visualization** | Training curves, bar chart, decision boundary |

**Key insight:** Different encodings capture different data patterns. The IQP encoding with ZZ interactions may better capture radial symmetry for circle classification.

In [ ]:
shutil.rmtree(WORKSPACE)
print("Done!")